In [ ]:
import pandas as pd, numpy as np

train_df = pd.read_csv('/data/playground-series-s5e11/train.csv')
test_df = pd.read_csv('/data/playground-series-s5e11/test.csv')
orig = pd.read_csv('/data/playground-series-s5e11loan_dataset_20000.csv')

print('Train Shape:', train_df.shape)
print('Test Shape:', test_df.shape)

In [ ]:
TARGET = 'loan_paid_back'  #boolean
CATS = ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']
BASE = [col for col in train_df.columns if col not in ['id', TARGET]]

train = train_df.copy()
test = test_df.copy()

In [ ]:
ORIG = []

for col in BASE:
    mean_map = orig.groupby(col)[TARGET].mean()
    new_mean_col_name = f"orig_mean_{col}"
    mean_map.name = new_mean_col_name
    
    train = train.merge(mean_map, on=col, how='left')
    test = test.merge(mean_map, on=col, how='left')
    ORIG.append(new_mean_col_name)

    new_count_col_name = f"orig_count_{col}"
    count_map = orig.groupby(col).size().reset_index(name=new_count_col_name)
    
    train = train.merge(count_map, on=col, how='left')
    test = test.merge(count_map, on=col, how='left')
    ORIG.append(new_count_col_name)

print(len(ORIG), 'Orig Features Created!!')

FEATURES = BASE + ORIG
print(len(FEATURES), 'Total Features.')